### Problem

In [41]:
public class HttpRequest {
    public string UserName { get; set; }
    public string Password { get; set; }
}

public class WebServer {
    public void Handle(HttpRequest request) {
        var authenticator = new Authenticator ();
        authenticator.Authenticate(request);
        
        var compressor = new Compressor();
        compressor.Compress(request);
        
        var logger = new Logger();
        logger.Log(request);
    }
}

public class Authenticator {
    public bool Authenticate(HttpRequest request) {
        var isValid = (request.UserName == "Admin" 
                       && request.Password == "Admin");
        
        Console.WriteLine("Authentication...");
        return isValid;
    }
}

public class Compressor {
    public void Compress(HttpRequest request) {
        Console.WriteLine("Compress...");
    }
}

public class Logger {
    public void Log(HttpRequest request) {
        Console.WriteLine("Log...");
    }
}

In [42]:
var request = new HttpRequest() {
    UserName = "Admin", 
    Password = "Admin"
};

var webServer = new WebServer ();
webServer.Handle(request);

Authentication...
Compress...
Log...


### Solution

In [43]:
public class HttpRequest {
    public string UserName { get; set; }
    public string Password { get; set; }
}

public class WebServer {
    private IHandler<HttpRequest> httpRequestHandler;
    
    public WebServer(IHandler<HttpRequest> httpRequestHandler) {
        this.httpRequestHandler = httpRequestHandler;
    }
    
    public void Handle(HttpRequest request) {
        httpRequestHandler.Handle(request);
    }
}

public interface IHandler<T> {
    void Handle(T request);
}

public abstract class Handler<T> : IHandler<T> {
    private Handler<T> next;
    
    public Handler (Handler<T> next) {
        this.next = next;
    }
    
    public void Handle (T request) {
        if (Process(request))
            return;
        
        if (next != null)
            next.Handle(request);
    }
    
    protected abstract bool Process(T request);
}

public class Authenticator : Handler<HttpRequest> {
    
    public Authenticator(Handler<HttpRequest> requestHandler) : base(requestHandler) {}
    
    protected override bool Process(HttpRequest request) {
        var isValid = (request.UserName == "Admin" 
                       && request.Password == "Admin");
        
        Console.WriteLine("Authentication...");
        return !isValid;
    }
}

public class Compressor : Handler<HttpRequest> {
    
    public Compressor(Handler<HttpRequest> requestHandler) : base(requestHandler) {}
    
    protected override bool Process(HttpRequest request) {
        Console.WriteLine("Compress...");
        return false;
    }
}

public class Logger : Handler<HttpRequest> {
    
    public Logger(Handler<HttpRequest> requestHandler) : base(requestHandler) {}
    
    protected override bool Process(HttpRequest request) {
        Console.WriteLine("Log...");
        return false;
    }
}

In [44]:
var requestAdmin = new HttpRequest () {
    UserName = "Admin",
    Password = "Admin"
};

var requestUser = new HttpRequest () {
    UserName = "User",
    Password = "User"
};

var webServer = new WebServer (
    new Authenticator(
        new Logger(
            new Compressor(null)
        )
    )
);
    
Console.WriteLine("-- Admin --");
webServer.Handle(requestAdmin);

Console.WriteLine("\n-- User --");
webServer.Handle(requestUser);

-- Admin --
Authentication...
Log...
Compress...

-- User --
Authentication...
